In [2]:
import pandas as pd

In [4]:
path = "C:/Users/eliza/Documents/Semester V/Sentiment Analysis/Kentucky - Metadata/meta-Kentucky.json"
business_df = pd.read_json(path, lines = True)

In [3]:
path = "C:/Users/eliza/Documents/Semester V/Sentiment Analysis/Kentucky - Review/review-Kentucky.json"
review_df = pd.read_json(path, lines = True)


In [5]:
# Aggregate business metadata by gmap_id before merging
business_agg = business_df.groupby("gmap_id", as_index=False).agg({
    "name": "first",  # Keep first occurrence (assumes same name per gmap_id)
    "address": "first",
    "description": "first",
    "latitude": "first",
    "longitude": "first",
    "category": lambda x: list(set(x.sum())) if x.apply(lambda v: isinstance(v, list)).any() else list(set(x)),
    "avg_rating": "mean",  # Use mean rating if multiple exist
    "num_of_reviews": "sum",
    "price": "first",
    "hours": "first",
    "MISC": "first",
    "state": "first",
    "relative_results": "first",
    "url": "first"
})

In [6]:
# Merge with aggregated business data
merged_df = review_df.merge(business_agg, on="gmap_id", how="left")



In [8]:
#remove rows/records where 'text' is null
merged_df = merged_df.dropna(subset = ['text'])
#check if null values in 'text' column still exist
print(merged_df['text'].isna().sum())

0


In [9]:
#delete pics 
merged_df = merged_df.drop(columns = ['pics'])


In [11]:
negative_df = merged_df[merged_df['rating'].isin([1,2])]
negative_df

,user_id,name_x,time,rating,text,resp,gmap_id,name_y,address,description,...,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
19,1.027216e+20,Marissa Roderick,1460471642644,1.0,Staff was smoking out front when I approached ...,None,0x8842fd94648d81cf:0xc781caba6e0e6def,Head Hunters 2,"Head Hunters 2, 805 Eastern Bypass # 5, Richmo...",None,...,-84.286938,"[Hair salon, Beauty salon]",4.0,8,None,None,None,None,"[0x8842fdf1371e8d85:0x3b51f1db435d68bc, 0x8842...",https://www.google.com/maps/place//data=!4m2!3...
41,1.112244e+20,Yeney Reina,1629654568888,1.0,"I have no idea, who is writing all this review...","{'time': 1629931463146, 'text': 'We would like...",0x8869093ade46a337:0xf0ccb63efc0f66e1,Regency Center,"Regency Center, 1550 Raydale Dr, Louisville, K...",None,...,-85.672269,"[Hospice, Retirement home, Rehabilitation cent...",3.3,90,None,"[[Thursday, Open 24 hours], [Friday, Open 24 h...",{'Accessibility': ['Wheelchair accessible entr...,Open 24 hours,"[0x88690d2773e65f31:0x5e1124736a74e66f, 0x8869...",https://www.google.com/maps/place//data=!4m2!3...
49,1.042934e+20,Mrs. Educate,1612561806226,1.0,Un professional upper & lower management...un ...,"{'time': 1612616841565, 'text': 'We would like...",0x8869093ade46a337:0xf0ccb63efc0f66e1,Regency Center,"Regency Center, 1550 Raydale Dr, Louisville, K...",None,...,-85.672269,"[Hospice, Retirement home, Rehabilitation cent...",3.3,90,None,"[[Thursday, Open 24 hours], [Friday, Open 24 h...",{'Accessibility': ['Wheelchair accessible entr...,Open 24 hours,"[0x88690d2773e65f31:0x5e1124736a74e66f, 0x8869...",https://www.google.com/maps/place//data=!4m2!3...
54,1.023253e+20,Jessica Jones,1601568816824,1.0,2 years ago my mom endured the worst treatment...,"{'time': 1601690226718, 'text': 'I would like ...",0x8869093ade46a337:0xf0ccb63efc0f66e1,Regency Center,"Regency Center, 1550 Raydale Dr, Louisville, K...",None,...,-85.672269,"[Hospice, Retirement home, Rehabilitation cent...",3.3,90,None,"[[Thursday, Open 24 hours], [Friday, Open 24 h...",{'Accessibility': ['Wheelchair accessible entr...,Open 24 hours,"[0x88690d2773e65f31:0x5e1124736a74e66f, 0x8869...",https://www.google.com/maps/place//data=!4m2!3...
59,1.109673e+20,Tracy,1512766536203,1.0,My father was in here twice. If I could give t...,None,0x8869093ade46a337:0xf0ccb63efc0f66e1,Regency Center,"Regency Center, 1550 Raydale Dr, Louisville, K...",None,...,-85.672269,"[Hospice, Retirement home, Rehabilitation cent...",3.3,90,None,"[[Thursday, Open 24 hours], [Friday, Open 24 h...",{'Accessibility': ['Wheelchair accessible entr...,Open 24 hours,"[0x88690d2773e65f31:0x5e1124736a74e66f, 0x8869...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7654679,1.182529e+20,Beverly Brown,1544330347794,1.0,Sucks,None,0x886fbbc9a3a19075:0x43473918860aada0,Shoney's,"Shoney's, 4710 Frederica St, Owensboro, KY 42301",Family-oriented chain serving an all-American ...,...,-87.124390,"[Hamburger restaurant, American restaurant, Br...",3.9,611,$,"[[Thursday, 6AM–3PM], [Friday, 6AM–9PM], [Satu...","{'Service options': ['Delivery', 'Takeout', 'D...",None,"[0x886fbbc65ad9ac7f:0x11fbc8b220b42760, 0x886f...",https://www.google.com/maps/place//data=!4m2!3...
7654681,1.120241e+20,heath price,1544730342515,2.0,Run always!!!! Run Away!!!!,None,0x886fbbc9a3a19075:0x43473918860aada0,Shoney's,"Shoney's, 4710 Frederica St, Owensboro, KY 42301",Family-oriented chain serving an all-American ...,...,-87.124390,"[Hamburger restaurant, American restaurant, Br...",3.9,611,$,"[[Thursday, 6AM–3PM], [Friday, 6AM–9PM], [Satu...","{'Service options': ['Delivery', 'Takeout', 'D...",None,"[0x886fbbc65ad9ac7f:0x11fbc8b220b42760, 0x886f...",https://www.google.com/maps/place//data=!4m2!3...
7654686,1.169558e+20,jonathan howe,1525099213358,2.0,Meh,None,0x886fbbc9a3a19075:0x43473918860aada0,Shoney's,"Shoney's, 4710 Frederica St, Owensboro, KY 42301",Family

In [10]:
merged_df.columns

Index(['user_id', 'name_x', 'time', 'rating', 'text', 'resp', 'gmap_id',
       'name_y', 'address', 'description', 'latitude', 'longitude', 'category',
       'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC', 'state',
       'relative_results', 'url'],
      dtype='object')

In [18]:
import ast

merged_df["category"] = merged_df["category"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Expand the list into separate columns
category_expanded = merged_df["category"].apply(pd.Series)

# Rename columns as 'category_1', 'category_2', etc.
category_expanded.columns = [f"category_{i+1}" for i in range(category_expanded.shape[1])]

# Concatenate with original dataset
merged_df = pd.concat([merged_df, category_expanded], axis=1)

# Drop original 'category' column
#cleaned_file.drop(columns=["category"], inplace=True)

merged_df

,user_id,name_x,time,rating,text,resp,gmap_id,name_y,address,description,...,category_3,category_4,category_5,category_6,category_7,category_8,category_9,category_10,category_11,category_12
0,1.035140e+20,Jed Weidner,1528824279086,5.0,Delicious organic fruits and vegetables. Homem...,None,0x88652157ed69241b:0x29312f00ef9611c0,Farm Fresh Produce,"Farm Fresh Produce, 540 Big Pond Rd, Trenton, ...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.163355e+20,Elif Demiral,1623168239098,5.0,Great farm to purchase fresh produce. They wer...,None,0x88652157ed69241b:0x29312f00ef9611c0,Farm Fresh Produce,"Farm Fresh Produce, 540 Big Pond Rd, Trenton, ...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.159915e+20,Thomas Hemingson,1625336163563,5.0,Wonderful family produce store. All home grown.,None,0x88652157ed69241b:0x29312f00ef9611c0,Farm Fresh Produce,"Farm Fresh Produce, 540 Big Pond Rd, Trenton, ...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.166816e+20,Matthew Smith,1500491517802,5.0,This is a nice little family run Amish produce...,None,0x88652157ed69241b:0x29312f00ef9611c0,Farm Fresh Produce,"Farm Fresh Produce, 540 Big Pond Rd, Trenton, ...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.104765e+20,tina randall,1604900546382,5.0,Amazing pies and baked goods. Jelly/jams are a...,None,0x88652157ed69241b:0x29312f00ef9611c0,Farm Fresh Produce,"Farm Fresh Produce, 540 Big Pond Rd, Trenton, ...",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7654704,1.178222e+20,Christopher Mcmillen,1475537745791,4.0,Great prices.,None,0x886fbbc9a3a19075:0x43473918860aada0,Shoney's,"Shoney's, 4710 Frederica St, Owensboro, KY 42301",Family-oriented chain serving an all-American ...,...,Breakfast restaurant,Family restaurant,Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7654705,1.043429e+20,James Williamson,1483401955450,5.0,Excellent buffet options,None,0x886fbbc9a3a19075:0x43473918860aada0,Shoney's,"Shoney's, 4710 Frederica St, Owensboro, KY 42301",Family-oriented chain serving an all-American ...,...,Breakfast restaurant,Family restaurant,Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7654706,1.021489e+20,Jeremy Glenn,1450540587623,1.0,Went here a few yrs. ago - everything was beyo...,None,0x886fbbc9a3a19075:0x43473918860aada0,Shoney's,"Shoney's, 4710 Frederica St, Owensboro, KY 42301",Family-oriented chain serving an all-American ...,...,Breakfast restaurant,Family restaurant,Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7654707,1.117471e+20,Robin Daniel,1456200741629,1.0,Their website states that they close at 11:00 ...,None,0x886fbbc9a3a19075:0x43473918860aada0,Shoney's,"Shoney's, 4710 Frederica St, Owensboro, KY 42301",Family-oriented chain serving an all-American ...,...,Breakfast restaurant,Family restaurant,Restaurant,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
import re

In [ ]:
healthcare_categories = {
    "medical clinic", "women's health clinic", "home health care service", "internist", "allergist",
    "diabetes center", "doctor", "family practice physician", "optometrist", "pharmacy", "dentist",
    "medical supply store", "orthopedic surgeon", "surgeon", "psychotherapist", "dental clinic",
    "orthodontist", "physical therapist", "chiropractor", "hospital", "sleep clinic", "optician",
    "cancer treatment center", "medical center", "physical therapy clinic", "medical laboratory",
    "public health department", "psychologist", "radiologist", "podiatrist", "nursing home",
    "medical group", "x-ray lab", "dermatologist", "pediatrician", "massage therapist",
    "periodontist", "obstetrician-gynecologist", "sports medicine clinic", "psychiatric hospital",
    "orthotics & prosthetics service", "physical fitness program", "ophthalmologist", "hospice",
    "retirement home", "rehabilitation center", "pediatric dentist", "teeth whitening service",
    "dental hygienist", "endodontist", "dental implants periodontist", "cosmetic dentist",
    "emergency dental service", "urgent care center", "wellness center", "walk-in clinic",
    "general practitioner", "children's hospital", "mental health clinic", "otolaryngologist", "eye care center", 
    "ophthalmology clinic","lasik surgeon","blood donation center" ,"Family practice physician",
    "Addiction treatment center","denture care center","gym","fitness center","personal trainer", 
    "Psychiatrist","Pharmacy", "Drug store","Vitamin & supplements store","weight loss service",
    "Pulmonologist","gymnastics center", "oral surgeon","Oral and maxillofacial surgeon","homeless service",
     "Alcoholism treatment program", "Homeless shelter", "child care agency","day care center",
     "Neurosurgeon","Vascular surgeon", "Cardiologist", "Laboratory", "Specialized clinic",
     "contact lenses supplier","massage spa","Reflexologist", "Rheumatologist", "Surgical center",
     "Mental health service", "Eating disorder treatment center", "Hair removal service", 
     "Skin care clinic","foot care","laser hair removal service","spa","medical spa",
     "sports nutrition store","pain control clinic","audiologist","plastic surgeon","pathologist",
     "Pediatric cardiologist","Sports medicine physician","Medical equipment supplier","nurse practitioner",
     "Psychotherapist","Counselor", "Family counselor", "Marriage or relationship counselor","facial spa",
     "Occupational medical physician","Urologist","Nephrologist","Endocrinologist",
     "Alternative medicine practitioner", "Holistic medicine practitioner","pain control clinic",
     "Psychologist","health","massage","Dialysis center", "Local medical services", "Medical Center",
     "vascular surgeon","hand surgeon","Gastroenterologist","Thai massage therapist","Hearing aid store", "Hearing aid repair service"
                                        
}
# Function to check if a business belongs to the healthcare category
def classify_healthcare(row):
    for i in range(1, 16):  # Assuming category_1 to category_15 exist
        category_col = f"category_{i}"
        if category_col in row and isinstance(row[category_col], str):  # Check if column exists and is a string
            for keyword in healthcare_categories:
                if re.search(rf"\b{keyword}\b", row[category_col], re.IGNORECASE):  # Match whole word
                    return "Healthcare"  # Assign healthcare label
    return "Other"  # Default category if no match

# Apply the function to create a new 'general_labelling' column
merged_df["general_labelling"] = merged_df.apply(classify_healthcare, axis=1)


In [ ]:
# Filter records where 'general_labelling' is "Healthcare"
healthcare_dataset = merged_df[merged_df["general_labelling"] == "Healthcare"]

# Define the file path to save the Healthcare dataset
kentucky_healthcare_dataset_path = "C:/Users/eliza/Documents/Semester V/Sentiment Analysis/kentucky_healthcare_dataset.csv"

# Save the filtered dataset to CSV
healthcare_dataset.to_csv(kentucky_healthcare_dataset_path, index=False)

In [23]:
path = "C:/Users/eliza/Documents/Semester V/Sentiment Analysis/kentucky_healthcare_dataset.csv"
kentucky_df = pd.read_csv(path)
kentucky_df

C:\Users\eliza\AppData\Local\Temp\ipykernel_38884\1846963646.py:2: DtypeWarning: Columns (9,15,18,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  kentucky_df = pd.read_csv(path)


,user_id,name_x,time,rating,text,resp,gmap_id,name_y,address,description,...,category_4,category_5,category_6,category_7,category_8,category_9,category_10,category_11,category_12,general_labelling
0,1.121767e+20,Nikkeya Dorsey,1627087084842,5.0,I’ve been a CNA 25 years. I have worked in 5 l...,"{'time': 1627435103602, 'text': ""Hi Nikkeya Do...",0x8869093ade46a337:0xf0ccb63efc0f66e1,Regency Center,"Regency Center, 1550 Raydale Dr, Louisville, K...",NaN,...,Nursing home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
1,1.162253e+20,Nicole Bussard,1627227499235,5.0,Regency Center has evolved leaps and bounds in...,"{'time': 1627420744989, 'text': 'Hello Nicole ...",0x8869093ade46a337:0xf0ccb63efc0f66e1,Regency Center,"Regency Center, 1550 Raydale Dr, Louisville, K...",NaN,...,Nursing home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
2,1.112244e+20,Yeney Reina,1629654568888,1.0,"I have no idea, who is writing all this review...","{'time': 1629931463146, 'text': 'We would like...",0x8869093ade46a337:0xf0ccb63efc0f66e1,Regency Center,"Regency Center, 1550 Raydale Dr, Louisville, K...",NaN,...,Nursing home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
3,1.162755e+20,Martinique Hall,1616769572755,5.0,Regency center is exactly what you look for in...,"{'time': 1617113877257, 'text': ""Hi Martinique...",0x8869093ade46a337:0xf0ccb63efc0f66e1,Regency Center,"Regency Center, 1550 Raydale Dr, Louisville, K...",NaN,...,Nursing home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
4,1.001592e+20,Dottie Coones,1617045859142,5.0,This is such a family oriented place. Social s...,"{'time': 1617113798358, 'text': ""Dottie Coones...",0x8869093ade46a337:0xf0ccb63efc0f66e1,Regency Center,"Regency Center, 1550 Raydale Dr, Louisville, K...",NaN,...,Nursing home,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153872,1.133809e+20,Kevin Kuhn,1543885174395,5.0,Professional staff from reception - all the wa...,NaN,0x8869766ab57d9453:0xc037b1aebe9f8ae6,Sara Cummins Aesthetic & Implant Dentistry,"Sara Cummins Aesthetic & Implant Dentistry, 92...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
153873,1.000319e+20,Lucinda Durham,1535403869678,5.0,"Friendly, pleasant office.",NaN,0x8869766ab57d9453:0xc037b1aebe9f8ae6,Sara Cummins Aesthetic & Implant Dentistry,"Sara Cummins Aesthetic & Implant Dentistry, 92...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
153874,1.047363e+20,Meg Fuqua,1568651713186,5.0,"Very professional, kind, and competent!",NaN,0x8869766ab57d9453:0xc037b1aebe9f8ae6,Sara Cummins Aesthetic & Implant Dentistry,"Sara Cummins Aesthetic & Implant Dentistry, 92...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
153875,1.088002e+20,Diane Riff,1520550044682,5.0,Wonderful care and atmosphere.,NaN,0x8869766ab57d9453:0xc037b1aebe9f8ae6,Sara Cummins Aesthetic & Implant Dentistry,"Sara Cummins Aesthetic & Implant Dentistry, 92...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Healthcare
